In [56]:
import numpy as np

In [57]:
class Softmax_Classifier:
    def __init__(self,epochs=1000,learning_rate=0.1,patience=20,min_del=1e-4):
        self.w=None
        self.b=None
        self.no_of_class=None
        self.epochs=epochs
        self.learning_rate=learning_rate
        self.patience=patience
        self.min_del=min_del

    def softmax(self,z):
        num=np.exp(z-np.max(z,axis=1,keepdims=True))
        return num/np.sum(num,axis=1,keepdims=True)
    
    def one_hot(self,y):
        one_hot=np.zeros((y.shape[0],self.no_of_class))
        for i in range(y.shape[0]):
            one_hot[i,y[i]]=1
        return one_hot
    
    def compute_loss(self,y_true,y_pred):
        m=y_true.shape[0]
        return -np.sum(y_true*np.log(y_pred+1e-4))/m
    

    def fit(self,X,y):
        # m-row n-column
        # y is m-row 1-column
        m,n=X.shape
        # number of classes
        self.no_of_class=np.max(y)+1
        # weight n*k(no. of class)
        self.w=np.zeros((n,self.no_of_class))
        # b 1*k
        self.b=np.zeros((1,self.no_of_class))
        epoch_no=0
        best_loss=float('inf')
        for epoch in range(self.epochs):
            # z m*k
            z=np.dot(X,self.w)+self.b
            # y_hat m*k
            y_hat=self.softmax(z)
            # m*c
            y_true=self.one_hot(y)

            loss=self.compute_loss(y_true,y_hat)
            if best_loss-loss>=self.min_del:
                best_loss=loss
                epoch_no=0
                
            else:
                epoch_no+=1
                
            if epoch_no>=self.patience:
                print(f"No improvement since {self.patience} epochs.Performing early stoping at {epoch}...")
                break


            # applying gradient descent
            self.w-=self.learning_rate*np.dot(X.T,(y_hat-y_true))/m
            self.b-=np.sum(self.learning_rate*(y_hat-y_true),keepdims=True)/m
        
    def predict(self,X):
        z=np.dot(X,self.w)+self.b
        prob=self.softmax(z)
        return np.argmax(prob,axis=1)
    
    def predict_prob(self,X):
        z=np.dot(X,self.w)+self.b
        return self.softmax(z)

    
        

In [58]:
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from time import time


X, y = make_classification(n_samples=10000,     
                           n_features=100,         
                           n_informative=30,     
                           n_redundant=10,
                           n_classes=5,          
                           random_state=42)

scaler = StandardScaler()
X = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)
# custom model
custom_model = Softmax_Classifier(epochs=1000, learning_rate=0.1,patience=30,min_del=0.001)
start = time()
custom_model.fit(X_train, y_train)
custom_time = time() - start
custom_acc = np.mean(custom_model.predict(X_test) == y_test)

# sklearn model
sklearn_model = LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=1000)
start = time()
sklearn_model.fit(X_train, y_train)
sklearn_time = time() - start
sklearn_acc = sklearn_model.score(X_test, y_test)

#Print
print("Custom Softmax Classifier:")
print(f" - Accuracy: {custom_acc:.4f}")
print(f" - Training Time: {custom_time:.2f} seconds\n")

print("Sklearn LogisticRegression:")
print(f" - Accuracy: {sklearn_acc:.4f}")
print(f" - Training Time: {sklearn_time:.2f} seconds")


No improvement since 30 epochs.Performing early stoping at 342...
Custom Softmax Classifier:
 - Accuracy: 0.5785
 - Training Time: 2.54 seconds

Sklearn LogisticRegression:
 - Accuracy: 0.5770
 - Training Time: 0.04 seconds


c:\Users\avata\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


In [59]:
custom_model.predict_prob(X_test)

array([[0.6786911 , 0.03382355, 0.10000227, 0.16572225, 0.02176084],
       [0.21382901, 0.04190716, 0.19071795, 0.2427534 , 0.31079248],
       [0.03032151, 0.63985169, 0.01325852, 0.07195761, 0.24461067],
       ...,
       [0.10836465, 0.33606695, 0.43920993, 0.05525866, 0.0610998 ],
       [0.0759175 , 0.0530696 , 0.25260778, 0.03352222, 0.58488289],
       [0.01706147, 0.27449905, 0.42383419, 0.01004779, 0.2745575 ]])

In [60]:
custom_model.w.shape

(100, 5)

In [61]:
custom_model.b

array([[2.92387642e-17, 2.92387642e-17, 2.92387642e-17, 2.92387642e-17,
        2.92387642e-17]])